In [ ]:
# @title 📊 數據整合平台 v15 - 穩定版啟動器
# @markdown ---
# @markdown ### **系統版本：v15.x.x (對應最新 GitHub 提交)**
# @markdown ### **核心架構：模듈화、高穩定性設計**
# @markdown 歡迎使用數據整合平台 v15！此版本基於強化的模組化工程構建，旨在提供前所未有的執行穩定性與更清晰的程式碼結構。
# @markdown - **智慧型程式碼更新**：啟動器會自動從 GitHub 拉取最新的程式碼，確保您永遠使用最新、最優化的版本。
# @markdown - **精準依賴管理**：透過 Poetry 精確鎖定套件版本，避免環境衝突，保障執行一致性。
# @markdown - **簡化參數配置**：所有可調整參數均集中於下方，方便您一站式完成設定。
# @markdown ---
# @markdown ### 📝 **執行設定**
# @markdown --- 
# @markdown #### **(0) 執行模式選擇**
# @markdown ✅ 勾選此項以將您的專案資料夾與 Google Drive 同步，並永久保存產出。
# @markdown ❌ 取消勾選則進入「本地測試模式」，檔案將儲存於此 Colab 的臨時空間，適合快速測試，但關閉後資料會遺失。
enable_google_drive_integration = True #@param {type:"boolean"}
# @markdown ---
# @markdown #### **主要路徑與檔案名稱**
project_folder = "MyTaifexDataProject" #@param {type:"string"}
database_name = "processed_data.duckdb" #@param {type:"string"}
log_name = "pipeline.log" #@param {type:"string"}
# @markdown --- 
# @markdown #### **特定檔案處理**
# @markdown *若要指定多個ZIP檔，請用逗號分隔，例如 `file1.zip,file2.zip`。留空則代表處理全部。*
target_zip_files = "" #@param {type:"string"}
# @markdown ---
# @markdown #### **(Developer) 開發者選項**
# @markdown ⚠️ 僅在需要深入分析問題時開啟。開啟後將啟用詳細日誌與硬體效能監控，日誌檔案會變得非常大。
debug_mode = False #@param {type:"boolean"}
# @markdown ---

print("--- 參數確認 ---")
print(f"Google Drive 整合模式: {'啟用' if enable_google_drive_integration else '停用 (本地測試模式)'}")
print(f"專案資料夾: {project_folder}")
print(f"輸出資料庫: {database_name}")
print(f"輸出日誌檔: {log_name}")
print(f"指定ZIP檔: {'全部' if not target_zip_files else target_zip_files}")
print("--------------------\n")

# ======================================================================
#         穩健執行區 (請勿修改此線以下內容)
# ======================================================================
import os
import sys
import shutil
from google.colab import drive

# --- Dynamic Path Configuration ---
if enable_google_drive_integration:
    base_path = "/content/drive/MyDrive"
    print(f"ℹ️ Google Drive 整合已啟用。專案資料夾將嘗試建立於: {base_path}")
else:
    base_path = "/content"
    print(f"ℹ️ 本地測試模式已啟用。專案資料夾將建立於 Colab 臨時空間: {base_path}")
    print("⚠️ 注意：在本地測試模式下，所有產出檔案將在 Colab 工作階段結束後遺失。")

# project_folder is from @param earlier in the cell
# This project_output_path is for user info and potential pre-checks by the notebook.
# main.py is assumed to use project_folder name to create outputs in its CWD or handle --no-gdrive.
project_output_path = os.path.join(base_path, project_folder) 
print(f"ℹ️ 完整專案輸出路徑預期為: {project_output_path}\n")

# 步驟 1: 掛載 Google Drive (如果啟用)
if enable_google_drive_integration:
    try:
        print("⏳ 正在請求掛載 Google Drive...")
        drive.mount('/content/drive', force_remount=True)
        print("✅ Google Drive 掛載成功！")
    except Exception as e:
        print(f"❌ Google Drive 掛載失敗: {e}")
        print("⚠️ Google Drive 掛載失敗！")
        print("\n🤔 發生了什麼事？")
        print("    看起來您可能在彈出的授權視窗中選擇了「取消」，或是網路連線出現了問題。")
        print("\n👉 您可以怎麼做？")
        print("    1. 請點擊上方儲存格左側的 ▶️ 按鈕，再次執行一次，並在彈出視窗中點擊「連線至 Google 雲端硬碟」並「允許」授權。")
        print("    2. 如果問題仍然存在，您可以嘗試一個終極解決方案：點擊上方選單的「執行階段」->「中斷並刪除執行階段」，然後從頭開始重新執行此筆記本。")
        print("    3. 或者，您可以在上一個儲存格中取消勾選「啟用 Google Drive 整合」，以「本地測試模式」先行體驗。")
else:
    print("ℹ️ Google Drive 整合已停用。所有操作將在本地臨時環境進行。")

try:
    # 步驟 2: 下載/更新專案程式碼
    print("\n⏳ 正在準備專案程式碼...")
    # Step 1: Change to /content, remove old repo, clone, and change into repo dir
    os.chdir('/content') # Change to /content first
    print(f"  - 目前工作目錄: {os.getcwd()}")

    REPO_NAME = 'taifexd-date' # Define repo name
    REPO_URL = 'https://github.com/LITHEBOW/taifexd-date.git' # Use the provided URL
    REPO_PATH = os.path.join('/content', REPO_NAME)

    if os.path.exists(REPO_NAME):
        print(f"  - 正在刪除舊的 '{REPO_NAME}' 目錄...")
        shutil.rmtree(REPO_NAME) # Use shutil.rmtree
        print(f"  - 舊目錄 '{REPO_NAME}' 已刪除。")
    else:
        print(f"  - '{REPO_NAME}' 目錄不存在，無需刪除。")

    print(f"  - 執行 git clone 下載專案 {REPO_URL} 至 {REPO_NAME}...")
    get_ipython().system(f"git clone -q {REPO_URL} {REPO_NAME}")

    print(f"  - 切換工作目錄至 {REPO_PATH}...")
    os.chdir(REPO_NAME) # Change into the cloned repository
    print(f"✅ 專案程式碼準備完成！目前工作目錄: {os.getcwd()}\n")

    # 步驟 3: 同步鎖定檔並安裝依賴
    print("\n⏳ 正在同步 lock 檔案並安裝依賴...")
    get_ipython().system("pip install poetry -q")
    # 修正為 --no-update，確保與 main 分支行為一致：只解決不一致，不更新套件版本
    print("  - 執行 poetry lock --no-update 來解決不一致問題...")
    get_ipython().system("poetry lock --no-update")
    print("  - 執行 poetry install 來安裝套件...")
    get_ipython().system("poetry install --no-root --without dev")
    print("✅ 所有精確版本的 Python 套件均已安裝完成！\n")

    # 步驟 4: 建構並執行指令
    print("\n🚀 即將啟動數據整合平台...")
    command_parts = [
        "poetry run python main.py",
        f"--project-folder-name='{project_folder}'",
        f"--database-name='{database_name}'",
        f"--log-name='{log_name}'",
        f"--zip-files='{target_zip_files}'"
    ]
    if not enable_google_drive_integration:
        command_parts.append("--no-gdrive")
    
    # Add debug flag if debug_mode is True
    if debug_mode:
        command_parts.append("--debug")

    command = " ".join(command_parts)

    print(f"  - 將執行的最終指令:")
    print(f"    {command}")
    print("-" * 20)

    get_ipython().system(command)

    print("-" * 20)
    print("🎉 執行完畢！")
    if enable_google_drive_integration:
        print(f"ℹ️  請檢查您的 Google Drive/{project_folder} 資料夾以確認產出檔案。")
    else:
        print(f"ℹ️  本地測試模式執行完畢。產出檔案位於 Colab 環境的 /content/{project_folder} (或程式碼中指定的相對路徑下)，關閉 Colab 後將清除。")
except Exception as e:
    print("\n❌ 哎呀！執行過程中似乎遇到了一個預期外的問題。")
    print("\n🤔 發生了什麼事？")
    print("    這可能是一個暫時的網路問題，或是程式碼的某個部分遇到了未曾預料的資料狀況。")
    print(f"\n    【技術性錯誤細節】: {e}")
    print("\n👉 您可以怎麼做？")
    log_dir_suggestion = os.path.join(project_folder, "99_logs") if 'project_folder' in locals() else "專案資料夾內的 99_logs"
    print(f"    1. 您可以先去專案日誌檔中尋找更詳細的線索，它通常位於您的 {log_dir_suggestion} 目錄中。")
    print("    2. 最常見的解決方法是：點擊上方選單的「執行階段」->「中斷並刪除執行階段」，然後從頭開始重新執行一次。這個動作可以解決大部分的暫時性問題。")
    print("    3. 如果問題重複出現，請考慮將【技術性錯誤細節】的內容回報給開發者。")
